In [2]:
import pandas as pd
import torch
import re
import string
import numpy as np
import streamlit as st
import faiss # хранение индексов
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from joblib import dump, load # Для сохранения/загрузки эмбэддингов

In [1]:
path = '/content/movies_filtered.csv' # ИЗМЕНИ ТУТ ПУТЬ!
a
df = pd.read_csv(path)

In [2]:
def clean(text):
    text = text.lower()  # Нижний регистр
    text = re.sub(r'\d+', ' ', text)  # Удаляем числа
    # text = text.translate(str.maketrans('', '', string.punctuation))  # Удаляем пунктуацию
    text = re.sub(r'\s+', ' ', text)  # Удаляем лишние пробелы
    text = text.strip()  # Удаляем начальные и конечные пробелы
    text = re.sub(r'\s+|\n', ' ', text) # Удаляет \n и \xa0
    # text = re.sub(r'\b\w{1,2}\b', '', text)  # Удаляем слова длиной менее 3 символов
    # Дополнительные шаги, которые могут быть полезны в данном контексте:
    # text = re.sub(r'\b\w+\b', '', text)  # Удаляем отдельные слова (без чисел и знаков препинания)
    # text = ' '.join([word for word in text.split() if word not in stop_words])  # Удаляем стоп-слова
    return text

for i, row in df.iterrows():
    df.at[i, 'description'] = clean(row['description'])

In [19]:
# pip install transformers sentencepiece

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

In [20]:
# применяем токенизатор:
# -≥ add_special_tokens = добавляем служебные токены (CLS=101, EOS=102)
# -≥ truncation = обрезаем по максимальной длине
# -≥ max_length = максимальная длина последовательности
tokenized = df['description'].apply((lambda x: tokenizer.encode(x,
                                                                      add_special_tokens=True,
                                                                      truncation=True,
                                                                      max_length=1024)))

In [21]:
max_len = 1024
# Делаю пэддинг чтобы добить до max_len последовательности
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
# И маску чтобы не применять self-attention на pad
attention_mask = np.where(padded != 0, 1, 0)

In [22]:
# Датасет для массивов
class BertInputs(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs, attention_masks):
        super().__init__()
        self.tokenized_inputs = tokenized_inputs
        self.attention_masks = attention_masks

    def __len__(self):
        return self.tokenized_inputs.shape[0]

    def __getitem__(self, idx):
        ids = self.tokenized_inputs[idx]
        ams = self.attention_masks[idx]

        return ids, ams

dataset = BertInputs(padded, attention_mask)

In [23]:
#DataLoader чтобы отправлять бачи в цикл обучения
loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
sample_ids, sample_ams = next(iter(loader))
print(sample_ids.shape, sample_ams.shape)

# shape BATCH_SIZE x MAX_LEN - что заходит в BERT

torch.Size([100, 1024]) torch.Size([100, 1024])


In [25]:
%%time

vectors_in_batch = []

# Iterate over all batches
for inputs, attention_masks in tqdm(loader):
    vectors_in_mini_batch = []  # Store vectors in mini-batch
    with torch.no_grad():
        last_hidden_states = model(inputs.cuda(), attention_mask=attention_masks.cuda())
        vector = last_hidden_states[0][:,0,:].detach().cpu().numpy()
        vectors_in_mini_batch.append(vector)

    vectors_in_batch.extend(vectors_in_mini_batch)

100%|██████████| 94/94 [01:13<00:00,  1.28it/s]

CPU times: user 1min 10s, sys: 145 ms, total: 1min 10s
Wall time: 1min 13s


In [16]:
import itertools

# Open the file and load the nested list
vectors_in_batch = load('vectors_in_batch.joblib')

# Convert the nested list to an unnested list
text_embeddings = list(itertools.chain.from_iterable(vectors_in_batch))

In [ ]:
# Сохранение эмбеддингов
dump(vectors_in_batch, 'vectors_in_batch.joblib')

In [17]:
len(vectors_in_batch)

94

In [9]:
len(text_embeddings)

9366